In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from enum import Enum 

app = FastAPI()

class Category(Enum):
    TOOLS = 'tools'
    CONSUMABLES = 'consumables'

class Item(BaseModel): # tells fastapi to validate data send through the endpoints to fit this schema
    name: str
    price: float
    count: int
    id: int
    category: Category 

items = {
    0: Item(name = 'Hammer', price = 9.99, count = 20, id = 0, category = Category.tools)
}

@app.get("/")
def index() -> dict[str, dict[int, Item]]:
    return {"items": items}

@app.get("/items/{item_id}") # path parameters uses an id to access a certain resource
def get_item(item_id: int) -> Item:
    if item_id not in items:
        raise HTTPException(status_code = 404)
    
    return items[item_id]

Selection = dict[ str, str | int | float | Category | None]

@app.get("/items/") # query parameter endpoints have urls like /items/?name=Hammer
def query_item_by_parameters( # this is how you can query items by specific criteria
    name: str | None = None, 
    price: float | None = None,
    count: int | None = None,
    category: Category | None = None
    ) -> dict[str, Selection]:
    def check_item(item: Item) -> bool:
        return all( # all checks if all are true
            (
                name is None or item.name == name,
                price is None or item.price == price,
                count is None or item.count == count,
                category is None or item.Category == Category
            )
        )
    selection = [item for item  in items.values() if check_item(item)]
    return selection

### Using Pydantic

In [13]:
from pydantic import BaseModel, field_validator

# When creating a class, you inherit from the BaseModel class
class User(BaseModel):
    name: str
    email: str
    id: int # define object properties and types

    # Custom Field Validation, must be a member of the class
    @field_validator("id")
    def validate_id(value):
        if value <= 0:
            raise ValueError("account value must be positive")
        return value


user = User(name = "Hello", email = "hello@gmail.com", id = 52)

print(user.name, user.email, user.id)

user_json = user.model_dump_json()
print(user_json)



Hello hello@gmail.com 52
{"name":"Hello","email":"hello@gmail.com","id":52}
